In [2]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import torch
from torch import nn

In [3]:
# Генерация данных
np.random.seed(42)
N = 1000
df = pd.DataFrame({
    'cat': np.random.choice(['A', 'B', 'C'], N),
    'num1': np.random.randn(N),
    'num2': np.random.rand(N) * 100,
    'num3': np.random.randint(0, 2, N),
    'num4': np.random.randn(N) * 5,
    'target_m': np.random.randint(0, 2, N),
    'target_k': np.random.randint(0, 2, N)
})

# Преобразование категориального признака
le = LabelEncoder()
df['cat_enc'] = le.fit_transform(df['cat'])
X = df[['cat_enc', 'num1', 'num2', 'num3', 'num4']]
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X.astype(np.float32))

# Обучающие выборки
y_m = df['target_m'].values
y_k = df['target_k'].values

X_train_m, _, y_train_m, _ = train_test_split(X_scaled, y_m, test_size=0.2)
X_train_k, _, y_train_k, _ = train_test_split(X_scaled, y_k, test_size=0.2)

In [4]:
# Модель
class SimpleClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(5, 8),
            nn.ReLU(),
            nn.Linear(8, 2)
        )

    def forward(self, x):
        return self.fc(x)

# Обучение моделей
def train_model(X, y):
    model = SimpleClassifier()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    X_tensor = torch.tensor(X, dtype=torch.float32)
    y_tensor = torch.tensor(y, dtype=torch.long)

    for _ in range(20):
        optimizer.zero_grad()
        out = model(X_tensor)
        loss = criterion(out, y_tensor)
        loss.backward()
        optimizer.step()
    return model

In [7]:
model_m = train_model(X_train_m, y_train_m)
model_k = train_model(X_train_k, y_train_k)

# Сохраняем всё
torch.save(model_m.state_dict(), "trained_models/model_m.pth")      # ← сохраняете весь объект модели
torch.save(model_k.state_dict(), "trained_models/model_k.pth")
# import pickle
# with open("le_cat.pkl", "wb") as f:
#     pickle.dump(le_cat, f)
# with open("scaler_X.pkl", "wb") as f:
#     pickle.dump(scaler, f)

In [5]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.model_selection import train_test_split
import pickle

# 1) Генерация синтетических данных
np.random.seed(42)
N = 1000
df = pd.DataFrame({
    'cat': np.random.choice(['A','B','C'], size=N),
    'num1': np.random.randn(N),
    'num2': np.random.rand(N) * 100,
    'num3': np.random.randint(0, 2, size=N),
    'num4': np.random.randn(N) * 5,
    'target_m': np.random.randint(0, 2, size=N),
    'target_k': np.random.randint(0, 2, size=N)
})

# 2) Кодируем категориальный признак
le_cat = LabelEncoder()
df['cat_enc'] = le_cat.fit_transform(df['cat'])  # "A"->0, "B"->1, "C"->2

# 3) Собираем матрицу признаков и масштабируем
X = df[['cat_enc','num1','num2','num3','num4']].astype(np.float32)  # shape = (N,5)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)  # scaler.mean_ и scaler.scale_ запоминаются внутри объекта

# 4) Целевые метки
y_m = df['target_m'].values   # (N,)
y_k = df['target_k'].values   # (N,)

# 5) Делим данные на train/test (для демонстрации; инференс пусть будет на всей выборке)
X_train_m, X_test_m, y_train_m, y_test_m = train_test_split(X_scaled, y_m, test_size=0.2, random_state=42)
X_train_k, X_test_k, y_train_k, y_test_k = train_test_split(X_scaled, y_k, test_size=0.2, random_state=42)

# 6) Простейший класс модели
class SimpleClassifier(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc = nn.Sequential(
            nn.Linear(5, 8),
            nn.ReLU(),
            nn.Linear(8, 2)
        )
    def forward(self, x):
        return self.fc(x)

# 7) Функция тренировки
def train_model(X_train, y_train):
    model = SimpleClassifier()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()
    X_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_tensor = torch.tensor(y_train, dtype=torch.long)

    for epoch in range(20):
        optimizer.zero_grad()
        outputs = model(X_tensor)
        loss = criterion(outputs, y_tensor)
        loss.backward()
        optimizer.step()
    return model

# 8) Обучаем две модели
model_m = train_model(X_train_m, y_train_m)
model_k = train_model(X_train_k, y_train_k)


In [1]:
# 9) Сохраняем PyTorch-модели
torch.save(model_m, "model_m.pth")
torch.save(model_k, "model_k.pth")

# # 10) Сохраняем препроцессоры через pickle
# with open("le_cat.pkl", "wb") as f:
#     pickle.dump(le_cat, f)

# with open("scaler_X.pkl", "wb") as f:
#     pickle.dump(scaler, f)

torch.save(model_m.state_dict(), "model_m.pth")      # ← сохраняете весь объект модели
torch.save(model_k.state_dict(), "model_k.pth")

NameError: name 'torch' is not defined